# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,316.505777,-0.501855,1234,316.003922,317.007633,632.007844,634.015265
6,142.646234,-0.701809,1234,141.944425,143.348043,283.888850,286.696085
7,649.885475,0.050152,1234,649.935627,649.835324,1299.871254,1299.670648
8,717.780245,-0.428277,1234,717.351968,718.208522,1434.703935,1436.417043
9,935.096223,0.796295,1234,935.892517,934.299928,1871.785035,1868.599856
10,206.815451,0.496131,1234,207.311581,206.319320,414.623163,412.638640
11,956.879472,0.178735,1234,957.058207,956.700737,1914.116415,1913.401474
12,107.267245,-0.403933,1234,106.863313,107.671178,213.726625,215.342357
13,570.738863,0.259296,1234,570.998159,570.479568,1141.996318,1140.959135
14,305.342332,0.780431,1234,306.122763,304.561901,612.245526,609.123801


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-316.003922,317.007633
6,-141.944425,143.348043
7,-649.935627,649.835324
8,-717.351968,718.208522
9,-935.892517,934.299928
10,-207.311581,206.319320
11,-957.058207,956.700737
12,-106.863313,107.671178
13,-570.998159,570.479568
14,-306.122763,304.561901


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-316.003922,317.007633
6,-141.944425,143.348043
7,-649.935627,649.835324
8,-717.351968,718.208522
9,-935.892517,934.299928
10,-207.311581,206.319320
11,-957.058207,956.700737
12,-106.863313,107.671178
13,-570.998159,570.479568
14,-306.122763,304.561901


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-316.00392211,  317.00763272],
       [-141.94442524,  143.34804268],
       [-649.93562707,  649.83532391],
       [-717.35196775,  718.20852157],
       [-935.89251746,  934.29992809],
       [-207.31158138,  206.31932011],
       [-957.05820735,  956.70073714],
       [-106.86331257,  107.6711783 ],
       [-570.99815921,  570.47956774],
       [-306.12276308,  304.5619006 ]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,630.007844,-634.015265
6,281.888850,-286.696085
7,1297.871254,-1299.670648
8,1432.703935,-1436.417043
9,1869.785035,-1868.599856
10,412.623163,-412.638640
11,1912.116415,-1913.401474
12,211.726625,-215.342357
13,1139.996318,-1140.959135
14,610.245526,-609.123801


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.316506,0.249072
6,0.142646,0.149096
7,0.649885,0.525076
8,0.717780,0.285862
9,0.935096,0.898147
10,0.206815,0.748065
11,0.956879,0.589368
12,0.107267,0.298034
13,0.570739,0.629648
14,0.305342,0.890216


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,316.505777,-0.501855
6,142.646234,-0.701809
7,649.885475,0.050152
8,717.780245,-0.428277
9,935.096223,0.796295
10,206.815451,0.496131
11,956.879472,0.178735
12,107.267245,-0.403933
13,570.738863,0.259296
14,305.342332,0.780431


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
